# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [2]:
import numpy as np

In [ ]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2025-04-11 23:48:58.245533: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-11 23:49:11.234222: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWJMQ5P/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWJMQ5P/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2025-04-11 23:49:50.842476: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure: 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [ ]:
# YOUR CODE HERE
from gensim.models import Word2Vec
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)
train_sentences, y_train = tfds.as_numpy(train_data)

word2vec = Word2Vec(
    sentences=train_sentences,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1,
    epochs=10
)


Let's reuse the functions of the previous exercise to convert your training and test data into something you can feed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad`:
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from finding errors too late and from letting them propagate through the entire notebook.

In [10]:
print("Word2Vec embedding size:", word2vec.wv.vector_size)

Word2Vec embedding size: 100


In [12]:
print( X_train_pad.shape , X_test_pad.shape)

(2500, 200) (2500, 200)


In [ ]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)
# dose not work but i understand what is doing

# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you are doing something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [15]:
# YOUR CODE HERE
from collections import Counter

most_common_class = Counter(y_train).most_common(1)[0][0]
baseline_predictions = np.full_like(y_test, fill_value=most_common_class)
baseline_accuracy = np.mean(baseline_predictions == y_test)

baseline_accuracy

0.492

# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [ ]:
# YOUR CODE HERE
from tensorflow.keras.layers import Masking, LSTM, Dense
from tensorflow.keras.models import Sequential

model = Sequential([

    Masking(mask_value=0., input_shape=(None, word2vec.wv.vector_size)),
    LSTM(20, activation='tanh'),

    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, None, 100)         0         
                                                                 
 lstm (LSTM)                 (None, 20)                9680      
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9,901
Trainable params: 9,901
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy will greatly depend on your training corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [ ]:
import numpy as np

X_train_emb = np.zeros((2500, 200, 100))
X_test_emb = np.zeros((len(X_test_pad), 200, 100))

for i, sentence in enumerate(X_train):
    for j, word in enumerate(sentence[:200]):
        if word in word2vec.wv:
            X_train_emb[i,j] = word2vec.wv[word]

for i, sentence in enumerate(X_test):
    for j, word in enumerate(sentence[:200]):
        if word in word2vec.wv:
            X_test_emb[i,j] = word2vec.wv[word]

In [ ]:
print("Training data shape:", X_train_emb.shape)
print("Test data shape:", X_test_emb.shape)

Training data shape: (2500, 200, 100)
Test data shape: (2500, 200, 100)


In [ ]:
# YOUR CODE HERE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense

model = Sequential([
    Masking(mask_value=0., input_shape=(200, 100)),
    LSTM(20, activation='tanh'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

history = model.fit(
    X_train_emb, y_train,
    validation_data=(X_test_emb, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[EarlyStopping(patience=3)]
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 200, 100)          0         
                                                                 
 lstm_1 (LSTM)               (None, 20)                9680      
                                                                 
 dense_2 (Dense)             (None, 10)                210       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9,901
Trainable params: 9,901
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
79/79 [==============================] - 22s 197ms/step - loss: 0.6932 - accuracy: 0.4908 - val_loss: 0.6932 - val_accuracy: 0.4920
Epoch 2/50
79/79 [=======================

❓ **Question** ❓ Evaluate your model on the test set

In [32]:
# YOUR CODE HERE
test_loss, test_accuracy = model.evaluate(X_test_emb, y_test, verbose=1)

79/79 [==============================] - 3s 32ms/step - loss: 0.6933 - accuracy: 0.4920


In [33]:
test_loss

0.6932827234268188

In [34]:
test_accuracy

0.492000013589859

# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. Let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that others have learned? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this: 

In [35]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is smaller (65 MB).

</details>

In [ ]:
# YOUR CODE HERE
import gensim.downloader as api

word2vec_transfer = api.load('glove-wiki-gigaword-50')

print(f"Loaded {word2vec_transfer.vector_size}-dimensional embeddings")
print(f"Vocabulary size: {len(word2vec_transfer.key_to_index)}")
print(f"Example embedding for 'movie': {word2vec_transfer['movie'][:10]}...")

[==================================================] 100.0% 66.0/66.0MB downloaded
Loaded 50-dimensional embeddings
Vocabulary size: 400000
Example embedding for 'movie': [ 0.30824   0.17223  -0.23339   0.023105  0.28522   0.23076  -0.41048
 -1.0035   -0.2072    1.4327  ]...


❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [37]:
# YOUR CODE HERE
print("'movie' in vocab:", 'movie' in word2vec_transfer)

'movie' in vocab: True


❓ Let's embed `X_train` and `X_test`, same as in the first question where we provided the functions to do so! (There is a slight difference in the `embed_sentence_with_TF` function that we will not dig into)

In [ ]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])

    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []

    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)

    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [39]:
# YOUR CODE HERE
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad_2 = pad_sequences(X_train_embed_2, maxlen=200, dtype='float32', padding='post', value=0)
X_test_pad_2 = pad_sequences(X_test_embed_2, maxlen=200, dtype='float32', padding='post', value=0)

print(X_train_pad_2.shape,X_test_pad_2.shape)

(2500, 200, 50) (2500, 200, 50)


❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training here could take some time. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [ ]:
# YOUR CODE HERE

model = Sequential([
    Masking(0., input_shape=(200, 50)),
    LSTM(20, activation='tanh'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_pad_2, y_train,
          validation_data=(X_test_pad_2, y_test),
          epochs=10,
          batch_size=32)

_, acc = model.evaluate(X_test_pad_2, y_test, verbose=0)
print(f"Test accuracy: {acc:.4f}")

Epoch 1/10
79/79 [==============================] - 22s 194ms/step - loss: 0.6902 - accuracy: 0.5336 - val_loss: 0.6772 - val_accuracy: 0.5744
Epoch 2/10
79/79 [==============================] - 14s 174ms/step - loss: 0.6601 - accuracy: 0.6148 - val_loss: 0.6518 - val_accuracy: 0.6208
Epoch 3/10
79/79 [==============================] - 16s 208ms/step - loss: 0.6203 - accuracy: 0.6632 - val_loss: 0.6391 - val_accuracy: 0.6400
Epoch 4/10
79/79 [==============================] - 15s 196ms/step - loss: 0.5982 - accuracy: 0.6880 - val_loss: 0.6084 - val_accuracy: 0.6708
Epoch 5/10
79/79 [==============================] - 16s 200ms/step - loss: 0.5760 - accuracy: 0.7048 - val_loss: 0.8364 - val_accuracy: 0.5376
Epoch 6/10
79/79 [==============================] - 16s 206ms/step - loss: 0.5684 - accuracy: 0.7100 - val_loss: 0.6722 - val_accuracy: 0.6472
Epoch 7/10
79/79 [==============================] - 16s 205ms/step - loss: 0.5531 - accuracy: 0.7248 - val_loss: 0.5951 - val_accuracy: 0.6872

In [41]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 66.080%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarded words that were not present more than a given number of times in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously